<a href="https://colab.research.google.com/github/Kashishsingla111/Samplingsubassignk/blob/main/Samplingassignk50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, StratifiedKFold
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
import numpy as np

In [2]:
data = pd.read_csv("/content/Creditcard_data.csv")

In [3]:
X = data.drop('Class', axis=1)
y = data['Class']

In [4]:
print("Number of instances before balancing:")
print(y.value_counts())

Number of instances before balancing:
0    763
1      9
Name: Class, dtype: int64


In [5]:
oversampler = RandomOverSampler(random_state=42)
X_balanced, y_balanced = oversampler.fit_resample(X, y)

print("\nNumber of instances after balancing:")
print(y_balanced.value_counts())


Number of instances after balancing:
0    763
1    763
Name: Class, dtype: int64


In [6]:
num_samples = 5

In [7]:
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'k-NN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

In [8]:
sampling_techniques = {
    'Stratified Sampling': StratifiedShuffleSplit(n_splits=num_samples, test_size=0.2, random_state=42),
    'Simple Sampling': None,  # Will be handled separately
    'Cluster Sampling': KMeans(n_clusters=num_samples, random_state=42).fit(X_balanced),
    'Systematic Sampling': None,  # Will be handled separately
    'Cross-Validation Sampling': StratifiedKFold(n_splits=num_samples, shuffle=True, random_state=42)
}

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
results = pd.DataFrame(index=models.keys(), columns=sampling_techniques.keys())
for model_name, model in models.items():
    for sampling_name, sampling in sampling_techniques.items():
        accuracies = []
        if 'Stratified' in sampling_name or 'Cross-Validation' in sampling_name:
            # Use StratifiedShuffleSplit or StratifiedKFold
            for train_index, test_index in sampling.split(X_balanced, y_balanced):
                X_train, X_test = X_balanced.iloc[train_index], X_balanced.iloc[test_index]
                y_train, y_test = y_balanced.iloc[train_index], y_balanced.iloc[test_index]

                model.fit(X_train, y_train)

                y_pred = model.predict(X_test)

                accuracy = accuracy_score(y_test, y_pred)
                accuracies.append(accuracy)
        elif 'Simple' in sampling_name:
            # Use Simple Sampling (train_test_split)
            for _ in range(num_samples):
                X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

                model.fit(X_train, y_train)

                y_pred = model.predict(X_test)

                accuracy = accuracy_score(y_test, y_pred)
                accuracies.append(accuracy)
        elif 'Systematic' in sampling_name:
            # Use Systematic Sampling (implement systematic_sampling function)
            for _ in range(num_samples):
                systematic_indices = np.arange(0, len(X_balanced), num_samples)
                X_train, y_train = X_balanced.iloc[systematic_indices], y_balanced.iloc[systematic_indices]

                model.fit(X_train, y_train)

                y_pred = model.predict(X_test)

                accuracy = accuracy_score(y_test, y_pred)
                accuracies.append(accuracy)
        else:
            # Use Cluster Sampling
            labels = sampling.labels_
            for label in np.unique(labels):
                cluster_indices = np.where(labels == label)[0]

                # Check if the cluster size is sufficient for splitting
                if len(cluster_indices) >= 2:
                    X_train, X_test, y_train, y_test = train_test_split(X_balanced.iloc[cluster_indices], y_balanced.iloc[cluster_indices], test_size=0.2, random_state=42)

                    model.fit(X_train, y_train)

                    y_pred = model.predict(X_test)

                    accuracy = accuracy_score(y_test, y_pred)
                    accuracies.append(accuracy)

        avg_accuracy = np.mean(accuracies)
        results.at[model_name, sampling_name] = avg_accuracy

In [10]:
for (model_name, sampling_name), accuracy in results.items():
    print(f'{model_name} with {sampling_name}: {accuracy:.4f}')

Logistic Regression with Stratified Sampling: 0.9248
Logistic Regression with Simple Sampling: 0.9183
Logistic Regression with Cluster Sampling: 0.9812
Logistic Regression with Systematic Sampling: 0.9307
Logistic Regression with Cross-Validation Sampling: 0.9207
Random Forest with Stratified Sampling: 1.0000
Random Forest with Simple Sampling: 1.0000
Random Forest with Cluster Sampling: 1.0000
Random Forest with Systematic Sampling: 1.0000
Random Forest with Cross-Validation Sampling: 1.0000
SVM with Stratified Sampling: 0.7098
SVM with Simple Sampling: 0.6699
SVM with Cluster Sampling: 0.8173
SVM with Systematic Sampling: 0.8020
SVM with Cross-Validation Sampling: 0.7169
k-NN with Stratified Sampling: 0.9843
k-NN with Simple Sampling: 0.9869
k-NN with Cluster Sampling: 0.9686
k-NN with Systematic Sampling: 0.9703
k-NN with Cross-Validation Sampling: 0.9810
Gradient Boosting with Stratified Sampling: 0.9980
Gradient Boosting with Simple Sampling: 1.0000
Gradient Boosting with Cluster 

In [16]:
from tabulate import tabulate

In [14]:
print("\nResults (Average Accuracy):")
print(tabulate(results, headers='keys', tablefmt='pretty'))


Results (Average Accuracy):
+---------------------+---------------------+--------------------+--------------------+---------------------+---------------------------+
|                     | Stratified Sampling |  Simple Sampling   |  Cluster Sampling  | Systematic Sampling | Cross-Validation Sampling |
+---------------------+---------------------+--------------------+--------------------+---------------------+---------------------------+
| Logistic Regression | 0.9248366013071895  | 0.9183006535947712 | 0.9811704384724187 | 0.9306930693069309  |    0.9206835958427086     |
|    Random Forest    |         1.0         |        1.0         |        1.0         |         1.0         |            1.0            |
|         SVM         | 0.7098039215686275  | 0.6699346405228758 | 0.8172921150810732 |  0.801980198019802  |    0.7169013179042109     |
|        k-NN         | 0.9843137254901961  | 0.9869281045751634 | 0.9685961810466761 | 0.9702970297029703  |    0.9809964641594343     |
|  Gr

In [19]:
results.to_csv('sampling_results.csv', index=True)